<a id="top"></a>

The purpose of this notebook is to train and test a predictive model for route 46A.  
The model should take in a variety of features (including time, day, weather) and should calculate the predicted journey time for the bus route.

***

# Import Packages

In [1]:
import pandas as pd
import sqlite3
import datetime
import calendar

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 

***

# Contents

- [1. Load in the Data](#load_in_the_data)

***

<a id="load_in_the_data"></a>
# 1. Load in the Data
[Back to contents](#contents)

#### Data Base Connection

In [2]:
# def function to create connection to db
def create_connection(db_file):
    """
    create a database connection to the SQLite database specified by db_file
    :param df_file: database file
    :return: Connection object or None
    """
    conn = None
    try: 
        conn = sqlite3.connect(db_file)
        return conn
    except 'Error' as e:
        print(e)
        
    return conn

In [3]:
# create connection to db
db_file = '/home/faye/Data-Analytics-CityRoute/dublinbus.db'
conn = create_connection(db_file)

## 1.1. Bus Data - Line 46A

For line 46A we will be using routes 46A_67 and 46A_74 as these are the routes with the largest number of stops.  
_67 is inbound, _74 is outbound

#### Query

In [4]:
# initialise query
#bus_query = """
#SELECT *
#FROM leavetimes
#WHERE TRIPID in 
#    (
#    SELECT TRIPID 
#    FROM trips
#    WHERE ROUTEID = '46A_74'
#    )
#"""

In [5]:
# initialise query
bus_query = """
SELECT *
FROM trips2
WHERE ROUTEID = '46A_74'
"""

In [6]:
# read in query to dataframe
df_outbound = pd.read_sql(bus_query, conn)

In [7]:
# drop unnecessary index feature and LASTUPDATE feature
df_outbound = df_outbound.drop(columns=['index', 'LASTUPDATE'])

#### Overview

In [8]:
# Print the number of rows and features in the dataset
num_rows = df_outbound.shape[0]
features = df_outbound.shape[1]
print(f"The dataset has {num_rows} rows with {features} features.")

The dataset has 37182 rows with 9 features.


In [9]:
# print first 5 rows
df_outbound.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,2018-02-07 00:00:00.000000,6253924,46A,46A_74,1,65768,60960,66075.0,60926.0
1,2018-02-07 00:00:00.000000,6262164,46A,46A_74,1,28877,24480,29657.0,24697.0
2,2018-03-11 00:00:00.000000,6397655,46A,46A_74,1,86458,83700,87190.0,83653.0
3,2018-03-11 00:00:00.000000,6392156,46A,46A_74,1,80158,77400,80661.0,77400.0
4,2018-03-11 00:00:00.000000,6392515,46A,46A_74,1,78358,75600,78412.0,75633.0


In [10]:
# print last 5 rows
df_outbound.tail(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
37177,2018-05-15 00:00:00.000000,6763274,46A,46A_74,1,39339,35040,39593.0,35040.0
37178,2018-05-15 00:00:00.000000,6753808,46A,46A_74,1,62027,57120,61827.0,57111.0
37179,2018-05-15 00:00:00.000000,6764906,46A,46A_74,1,49598,45120,49279.0,45157.0
37180,2018-05-15 00:00:00.000000,6762038,46A,46A_74,1,65510,60480,66254.0,60456.0
37181,2018-05-14 00:00:00.000000,6763192,46A,46A_74,1,65330,60300,64953.0,60273.0


#### Missing Values

In [11]:
# print number of missing values for each feature
df_outbound.isnull().sum()

DAYOFSERVICE          0
TRIPID                0
LINEID                0
ROUTEID               0
DIRECTION             0
PLANNEDTIME_ARR       0
PLANNEDTIME_DEP       0
ACTUALTIME_ARR     1485
ACTUALTIME_DEP     1568
dtype: int64

In [12]:
# drop rows with missing values
df_outbound.dropna(how='any', inplace=True)

##### Datatypes

In [13]:
# print data types of each feature
df_outbound.dtypes

DAYOFSERVICE        object
TRIPID              object
LINEID              object
ROUTEID             object
DIRECTION           object
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
dtype: object

In [14]:
# select date features
datetime_features = df_outbound[[
    'DAYOFSERVICE'
]].columns

for c in datetime_features:
    df_outbound[c] = df_outbound[c].astype('datetime64')

In [15]:
# select categorical features
categorical_features = df_outbound[[
    'TRIPID', 'LINEID', 'ROUTEID', 'DIRECTION'
]].columns

for c in categorical_features:
    df_outbound[c] = df_outbound[c].astype('category')

In [16]:
# select continuous features
continuous_features = df_outbound[[
    'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP',
    'ACTUALTIME_ARR', 'ACTUALTIME_DEP'
]].columns

for c in continuous_features: 
    df_outbound[c] = df_outbound[c].astype('int64')

In [17]:
# print data types of each feature
df_outbound.dtypes

DAYOFSERVICE       datetime64[ns]
TRIPID                   category
LINEID                   category
ROUTEID                  category
DIRECTION                category
PLANNEDTIME_ARR             int64
PLANNEDTIME_DEP             int64
ACTUALTIME_ARR              int64
ACTUALTIME_DEP              int64
dtype: object

#### Add Target Feature - ACTUALTIME_TRAVEL

In [18]:
# sort sequentially by TRIPID, DAYOFSERVICE, PROGRNUMBER
df_outbound = df_outbound.sort_values(['DAYOFSERVICE', 'TRIPID'])

In [19]:
# calculate planned travel time
#df_outbound['PLANNEDTIME_TRAVEL'] = (df_outbound['PLANNEDTIME_ARR'] - df_outbound['PLANNEDTIME_DEP'])

In [20]:
# calculate planned travel time
df_outbound['ACTUALTIME_TRAVEL'] = (df_outbound['ACTUALTIME_ARR'] - df_outbound['ACTUALTIME_DEP'])

#### Add MONTH feature

In [21]:
# add feature MONTHOFSERVICE
df_outbound['MONTHOFSERVICE'] = df_outbound['DAYOFSERVICE'].dt.strftime('%B')

#### Add DAYOFWEEK feature

In [22]:
# add feature DAYOFWEEK
df_outbound['DAYOFWEEK'] = [calendar.day_name[val.weekday()] for val in df_outbound['DAYOFSERVICE']]

#### Add datetime feature - for weather query

I will create a datetime feature, 'dt', that combines the DAYOFSERVICE with the hour of departure. This will be used to join the bus data with the weather data.

In [23]:
# initialise 'dt' feature
df_outbound['dt'] = 0

In [24]:
# create datetime feature combining DAYOFSERVICE and ACTUALTIME_DEP
for i in range(df_outbound.shape[0]):
    df_outbound.iloc[i, df_outbound.columns.get_loc('dt')] = datetime.datetime.fromtimestamp(df_outbound['DAYOFSERVICE'].iloc[i].timestamp() + df_outbound['ACTUALTIME_DEP'].iloc[i])

In [25]:
# remove minutes and seconds from dt feature
# new_date = old_date.replace(minute=0, second=0)
for i in range(df_outbound.shape[0]):
    df_outbound.iloc[i, df_outbound.columns.get_loc('dt')] = df_outbound['dt'].iloc[i].replace(minute=0, second=0)

In [26]:
# convert dt to datetime
df_outbound['dt'] = pd.to_datetime(df_outbound['dt'])

In [27]:
# print first 5 rows
df_outbound.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_TRAVEL,MONTHOFSERVICE,DAYOFWEEK,dt
35577,2018-01-01,5955569,46A,46A_74,1,50128,46200,49566,46227,3339,January,Monday,2018-01-01 12:00:00
35629,2018-01-01,5955571,46A,46A_74,1,60188,56400,59006,56346,2660,January,Monday,2018-01-01 15:00:00
35379,2018-01-01,5955573,46A,46A_74,1,69726,66000,69222,65929,3293,January,Monday,2018-01-01 18:00:00
35380,2018-01-01,5955575,46A,46A_74,1,77691,74700,77630,74693,2937,January,Monday,2018-01-01 20:00:00
35333,2018-01-01,5955577,46A,46A_74,1,86691,83700,86381,83679,2702,January,Monday,2018-01-01 23:00:00


#### Add HOUR feature

In [28]:
df_outbound['HOUR'] = df_outbound.dt.dt.hour

#### Check that ACTUALTIME_DEP < ACTUALTIME_ARR

In [29]:
# create a dataframe for check 2
check_2 = df_outbound[['ACTUALTIME_ARR','ACTUALTIME_DEP']][df_outbound['ACTUALTIME_DEP']>df_outbound['ACTUALTIME_ARR']]
print(f"Number of rows failing the check: {check_2.shape[0]}")
check_2

Number of rows failing the check: 1


,ACTUALTIME_ARR,ACTUALTIME_DEP
22613,73720,73729


In [30]:
# drop rows failing check
df_outbound.drop(index=22613, inplace=True)

## 1.2. Weather Data


In [31]:
# create dt tuple
dt_tuple = tuple(df_outbound.dt.dt.strftime('%Y-%m-%d %H:%M:%S.%f').to_list())

In [32]:
# initialise query
query = f"""
SELECT *
FROM weather
WHERE dt IN {dt_tuple}
"""

In [33]:
# read in query to dataframe
df_weather = pd.read_sql(query, conn)

In [34]:
# remove unnecessary index column
df_weather = df_weather.drop(columns='index')

In [35]:
# print first 5 rows
df_weather

,dt,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,clouds_all,weather_id,weather_main,weather_description
0,2018-01-01 08:00:00.000000,4.05,991,87,6.69,240,0.0,75,803,Clouds,broken clouds
1,2018-01-01 09:00:00.000000,4.08,992,87,7.72,240,0.0,40,802,Clouds,scattered clouds
2,2018-01-01 10:00:00.000000,5.02,992,81,9.77,240,0.0,40,802,Clouds,scattered clouds
3,2018-01-01 11:00:00.000000,6.01,994,81,12.35,260,0.0,40,802,Clouds,scattered clouds
4,2018-01-01 12:00:00.000000,7.01,994,76,12.35,270,0.0,75,803,Clouds,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
6256,2018-12-31 17:00:00.000000,9.47,1034,71,3.60,240,0.0,75,803,Clouds,broken clouds
6257,2018-12-31 18:00:00.000000,9.19,1034,75,4.60,250,0.0,75,803,Clouds,broken clouds
6258,2018-12-31 19:00:00.000000,9.65,1034,76,4.10,260,0.0,75,803,Clouds,broken clouds
6259,2018-12-31 20:00:00.000000,9.27,1034,81,4.10,250,0.0,75,803,Clouds,broken clouds


In [36]:
# print datatypes
df_weather.dtypes

dt                      object
temp                   float64
pressure                 int64
humidity                 int64
wind_speed             float64
wind_deg                 int64
rain_1h                float64
clouds_all               int64
weather_id               int64
weather_main            object
weather_description     object
dtype: object

In [37]:
# convert 'dt' to datetime
df_weather['dt'] = df_weather['dt'].astype('datetime64')

## 1.3. Create New Dataframe - Combine Bus and Weather

#### Overview of Merged Data

In [38]:
# print shape of bus data
df_outbound.shape

(34208, 14)

In [39]:
# print shape of weather data
df_weather.shape

(6261, 11)

In [40]:
# merge bus and weather data based on dt feature
df = pd.merge(df_outbound, df_weather, on='dt')

In [41]:
# print the number of rows and features in the dataset
num_rows = df.shape[0]
features = df.shape[1]
print(f"The dataset has {num_rows} rows with {features} features.")

The dataset has 34326 rows with 24 features.


In [42]:
# print the first 5 rows
print("The first 5 Rows are:")
df.head(5)

The first 5 Rows are:


,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_TRAVEL,...,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,clouds_all,weather_id,weather_main,weather_description
0,2018-01-01,5955569,46A,46A_74,1,50128,46200,49566,46227,3339,...,7.01,994,76,12.35,270,0.0,75,803,Clouds,broken clouds
1,2018-01-01,5958391,46A,46A_74,1,48720,45000,48481,44949,3532,...,7.01,994,76,12.35,270,0.0,75,803,Clouds,broken clouds
2,2018-01-01,5961258,46A,46A_74,1,46920,43200,46127,43229,2898,...,7.01,994,76,12.35,270,0.0,75,803,Clouds,broken clouds
3,2018-01-01,5964615,46A,46A_74,1,47520,43800,47567,44053,3514,...,7.01,994,76,12.35,270,0.0,75,803,Clouds,broken clouds
4,2018-01-01,5972013,46A,46A_74,1,49528,45600,48805,45622,3183,...,7.01,994,76,12.35,270,0.0,75,803,Clouds,broken clouds


In [43]:
# print the last 5 rows
print("The last 5 Rows are:")
df.tail(5)

The last 5 Rows are:


,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_TRAVEL,...,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,clouds_all,weather_id,weather_main,weather_description
34321,2018-12-31,8580475,46A,46A_74,1,76912,73800,77395,73764,3631,...,9.27,1034,81,4.1,250,0.0,75,803,Clouds,broken clouds
34322,2018-12-31,8585311,46A,46A_74,1,76709,72900,76780,72873,3907,...,9.27,1034,81,4.1,250,0.0,75,803,Clouds,broken clouds
34323,2018-12-31,8589246,46A,46A_74,1,75809,72000,75529,72002,3527,...,9.27,1034,81,4.1,250,0.0,75,803,Clouds,broken clouds
34324,2018-12-31,8586161,46A,46A_74,1,81412,78300,81006,78272,2734,...,9.31,1034,81,5.1,260,0.0,75,803,Clouds,broken clouds
34325,2018-12-31,8588976,46A,46A_74,1,79612,76500,80039,76491,3548,...,9.31,1034,81,5.1,260,0.0,75,803,Clouds,broken clouds


In [44]:
# print number of unique values for each feature
df.nunique()

DAYOFSERVICE             360
TRIPID                 10925
LINEID                     1
ROUTEID                    1
DIRECTION                  1
PLANNEDTIME_ARR         6437
PLANNEDTIME_DEP          255
ACTUALTIME_ARR         26095
ACTUALTIME_DEP         21292
ACTUALTIME_TRAVEL       3867
MONTHOFSERVICE            12
DAYOFWEEK                  7
dt                      6238
HOUR                      19
temp                    2007
pressure                  63
humidity                  63
wind_speed                82
wind_deg                  45
rain_1h                    8
clouds_all                80
weather_id                20
weather_main               8
weather_description       21
dtype: int64

In [45]:
# drop constant columns
df = df.drop(columns=['LINEID','ROUTEID','DIRECTION'])

In [46]:
# print the datatype of each feature
df.dtypes

DAYOFSERVICE           datetime64[ns]
TRIPID                       category
PLANNEDTIME_ARR                 int64
PLANNEDTIME_DEP                 int64
ACTUALTIME_ARR                  int64
ACTUALTIME_DEP                  int64
ACTUALTIME_TRAVEL               int64
MONTHOFSERVICE                 object
DAYOFWEEK                      object
dt                     datetime64[ns]
HOUR                            int64
temp                          float64
pressure                        int64
humidity                        int64
wind_speed                    float64
wind_deg                        int64
rain_1h                       float64
clouds_all                      int64
weather_id                      int64
weather_main                   object
weather_description            object
dtype: object

In [47]:
# print the number of null values for each feature
df.isna().sum()

DAYOFSERVICE           0
TRIPID                 0
PLANNEDTIME_ARR        0
PLANNEDTIME_DEP        0
ACTUALTIME_ARR         0
ACTUALTIME_DEP         0
ACTUALTIME_TRAVEL      0
MONTHOFSERVICE         0
DAYOFWEEK              0
dt                     0
HOUR                   0
temp                   0
pressure               0
humidity               0
wind_speed             0
wind_deg               0
rain_1h                0
clouds_all             0
weather_id             0
weather_main           0
weather_description    0
dtype: int64

In [48]:
# print range of target feature - ACTUALTIME_TRAVEL
target_range = df.ACTUALTIME_TRAVEL.max() - df.ACTUALTIME_TRAVEL.min()
print(f"{target_range} seconds")
print(datetime.timedelta(seconds=int(target_range)))

12119 seconds
3:21:59


In [49]:
df.ACTUALTIME_TRAVEL.max()

12138

In [50]:
df.ACTUALTIME_TRAVEL.mean()

4365.070820952048

In [51]:
df[df['ACTUALTIME_TRAVEL'] == 19]

,DAYOFSERVICE,TRIPID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_TRAVEL,MONTHOFSERVICE,DAYOFWEEK,dt,...,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,clouds_all,weather_id,weather_main,weather_description
33935,2018-12-26,8580585,78134,75600,80047,80028,19,December,Wednesday,2018-12-26 22:00:00,...,9.09,1025,87,2.6,190,0.0,75,803,Clouds,broken clouds


In [52]:
df.drop(index=[33935,6020, 27041]).ACTUALTIME_TRAVEL.min()

1944

In [53]:
df[df['ACTUALTIME_TRAVEL'] == 1944]

,DAYOFSERVICE,TRIPID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,ACTUALTIME_TRAVEL,MONTHOFSERVICE,DAYOFWEEK,dt,...,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,clouds_all,weather_id,weather_main,weather_description
15748,2018-06-10,6878234,57998,54000,58795,56851,1944,June,Sunday,2018-06-10 16:00:00,...,19.35,1015,60,4.63,100,0.0,40,802,Clouds,scattered clouds


***

[Back to top](#top)